# FAQ

This Section contains Frequently Asked Questions from our user community. The information in this Section contains feedback from the CASA development team, but its content is not officially validated and is at the user's discretion. We welcome [feedback](https://casa.nrao.edu/index_contact.shtml).






## Astropy in monolithic CASA

While the CASA team recommends to install the pip-wheel version of CASA into a customized Python environment (see [modular installation instructions](introduction.ipynb#Modular-Packages)), some use cases require packages like astropy to be installed into the monolithic CASA version.

To add astropy to monolithic CASA, one can use the pip3 that comes with casa. One would need to give the absolute path to pip3 inside the CASA distribution, e.g. /[full path]/casa-6.6.3-22-py3.8..el7/bin/pip3 install astropy. If not, then the OS will use the first pip3 it finds in PATH, which would install astropy in the wrong place and potentially for the wrong python version, etc. Astropy can then be run using the python syntax (see [CASA examples](../examples/community/casa6_demo.ipynb)).

Keep in mind that in this case one has installed astropy into the CASA installation, so it is now available for everyone using that installation. For a shared installation with multiple users, one could install astropy twice into different paths and then modifying one of them.

If the user running pip3 does not have permission to write inside the casa folder, pip will likely fall back to a --user installation and put the packages in \$HOME/.local. This will be apparent from the messages pip prints out. If this happens, then starting casa with --user-site should find the packages installed in $HOME/.local. However, this does not allow a good separation between the environments, so is not recommended and is at the user's discretion.



## Running pipeline in non-interactive / batch mode

Production pipeline runs are done as HPC scheduler batch jobs (via Slurm). However, when users wish to run the pipeline in non-interactive / batch mode, for example on a high-performance cluster, they may run into the problem that the CASA plotms needs x11 to make plots (while other pipeline tasks use matplotlib). The error message reads as follows:
```
casa.qtapp: cannot connect to X server localhost:10.0
```

Because of this, it is suggested to run a pipeline in non-interactive mode using a virtual buffer, Xvfb. The typical exe cmds sent to hpc scheduler are something like this:

```
xvfb-run -d ${casabin}/mpicasa ${casain}/casa —nologger —nogui -agg -c casa_script.py
```

Note that Ubuntu machines have an old xvfb-run version, so it would be "xvfb-run -a”.

For running the Pipeline from Singularity (“apptainer”) or Docker instances, you can 1) either "bake" xvfb into the container image (e.g. in Docker files, add the installation of xvfb), or 2) do X11 forwarding to the host (needs to open up ports and increase security risk).  Still, one needs to start things from "xvfb-run”.
